Alright — here’s a **comprehensive, in-depth set of notes** for your DBMS course module *Concurrency Control/2* based on your transcript.
I’ve covered **every topic** mentioned, including prevention, detection, recovery, and the time-based protocol.
I’ll also structure it so you can directly use it as study material.

---

# **DBMS – Concurrency Control/2**

## **Learning Outcomes**

1. Understand **deadlock** and strategies for **detection, prevention, and recovery**.
2. Learn **timestamp-based deadlock prevention schemes** (Wait-Die, Wound-Wait).
3. Understand **wait-for graph** for deadlock detection.
4. Learn **victim selection & rollback** strategies in recovery.
5. Introduce a **simple timestamp-based protocol** for deadlock-free execution.

---

## **1. Deadlock in Database Systems**

### **Definition**

* A **deadlock** occurs when a set of transactions are each **waiting for another transaction in the set** to release a lock, resulting in **no progress**.
* The simplest case: 2 transactions wait on each other.
* Can also occur with multiple transactions in a circular wait.

### **Example**

* T1 waits for a data item locked by T2
* T2 waits for a data item locked by T1
* Neither can proceed → deadlock.

---

## **2. Deadlock Prevention**

Goal: Ensure the system **never enters a deadlock state**.

### **Strategies**

1. **Pre-locking all data items** before starting a transaction

   * Requires a transaction to lock *all* needed items before it begins.
   * Impractical for large numbers of items → **reduces concurrency**.

2. **Partial ordering of data items**

   * Impose a fixed ordering of locks.
   * Transactions can request locks only in this order.
   * Prevents cycles in the wait-for graph.

3. **Timestamp-based prevention schemes**

   * Use **timestamps** to determine which transactions wait and which are aborted.
   * Timestamp = unique identifier for transaction order (based on start time, not necessarily real clock time).

---

## **3. Timestamp-Based Deadlock Prevention Schemes**

### **Timestamp Basics**

* `TS(Ti)` = timestamp of transaction `Ti`.
* Smaller `TS` → older transaction.
* Two main schemes:

---

#### **A. Non-Preemptive: Wait-Die Scheme**

* **Older transactions** may **wait** for younger ones.
* **Younger transactions** requesting a lock held by older transactions are **aborted** (killed) immediately.

**Rules:**
If `TS(Tn) < TS(Tk)` → **Tn waits**.
If `TS(Tn) > TS(Tk)` → **Tn is aborted** and restarted **with the same timestamp** after a random delay.

**Key Points:**

* “Die” = younger transaction killed.
* Avoids deadlock.
* Restart with same timestamp to avoid *becoming even younger* each time.

---

#### **B. Preemptive: Wound-Wait Scheme**

* **Older transactions** preempt (wound) younger ones.
* **Younger transactions** may wait for older ones.

**Rules:**
If `TS(Tn) < TS(Tk)` → **Tn kills Tk** (preemption).
If `TS(Tn) > TS(Tk)` → **Tn waits**.

**Analogy:**

* Older person enters the room → younger gives up the chair.
* Younger person enters → waits until older person leaves.

**Key Points:**

* “Wound” = older transaction forces younger to abort.
* Prevents deadlock, reduces wasted work for older transactions.

---

## **4. Other Prevention Methods**

* **Timeout-based**: A transaction waits for a lock only up to a fixed time limit.
  If lock not granted → transaction aborts and restarts.
* Guarantees no deadlock, but **starvation possible** (a transaction may never get the resource).

---

## **5. Deadlock Detection**

If prevention is not used, system must detect deadlock dynamically.

### **Wait-For Graph (WFG)**

* **Vertices**: Transactions
* **Edges**: Ti → Tj means Ti is waiting for a resource held by Tj.
* **Detection Rule**: Deadlock exists **iff** WFG contains a **cycle**.
* WFG is **continuously updated** as lock requests and releases happen.

---

## **6. Deadlock Recovery**

When deadlock detected:

1. **Select a victim transaction** to rollback.

   * Goal: **minimum cost** (minimize wasted work).
   * Cost factors:

     * How much work has been done.
     * How many resources held.
     * How many rollbacks already suffered (to avoid starvation).

2. **Rollback Options**:

   * **Total rollback**: abort entire transaction.
   * **Partial rollback**: rollback only to a point (lock point) that breaks the deadlock.

3. **Avoiding Starvation**:

   * Increase victim cost for transactions that have been rolled back more often.
   * Ensures every transaction eventually proceeds.

---

## **7. Timestamp-Based Concurrency Protocol (Deadlock-Free)**

A different use of timestamps — not just for deadlock prevention but also for **controlling read/write operations**.

### **For each data item Q:**

* **W-timestamp(Q)** = timestamp of most recent **write** to Q.
* **R-timestamp(Q)** = timestamp of most recent **read** of Q.

---

### **Read(Q) by Ti**:

1. If `TS(Ti) < W-timestamp(Q)`:

   * Ti needs an **obsolete value** (written after Ti started) → **Abort Ti**.
2. Else:

   * Allow read.
   * Update `R-timestamp(Q) = max(R-timestamp(Q), TS(Ti))`.

---

### **Write(Q) by Ti**:

1. If `TS(Ti) < R-timestamp(Q)`:

   * Ti is older than the latest read of Q → writing would violate serializability → **Abort Ti**.
2. If `TS(Ti) < W-timestamp(Q)`:

   * Ti is older than the latest write → would write obsolete value → **Abort Ti**.
3. Else:

   * Allow write.
   * Update `W-timestamp(Q) = TS(Ti)`.

---

### **Properties:**

* **Guarantees serializability** (transactions ordered by timestamps).
* **No waiting** → **No deadlock**.
* Can lead to:

  * **Not cascade-free** schedules.
  * Possibly **non-recoverable** schedules.

---

## **8. Advantages & Limitations**

**Advantages:**

* Simple logic.
* No deadlock (no waiting).
* Easy to implement.

**Limitations:**

* May cause **more aborts**.
* May produce schedules that are **not recoverable**.
* May not be cascade-free.

---

## **Summary Table: Deadlock Handling Approaches**

| **Method**         | **Idea**                                  | **Deadlock?** | **Starvation?** |
| ------------------ | ----------------------------------------- | ------------- | --------------- |
| Wait-Die           | Older waits, younger dies                 | No            | Possible        |
| Wound-Wait         | Older wounds younger, younger waits       | No            | Possible        |
| Timeout            | Abort after waiting too long              | No            | Possible        |
| Wait-For Graph     | Detect cycle, then abort victim           | Possible      | Avoidable       |
| Timestamp Protocol | Abort conflicting ops based on timestamps | No            | Possible        |